In [1]:

from sklearn.base import TransformerMixin, BaseEstimator

from collections import Counter , defaultdict

import pandas as pd

pd.set_option('display.max_columns', None)


from pandas import Series as s , DataFrame as df
import numpy as np

import seaborn as sns
# import matplotlib.pyplot as plt

# To ignore warnings
import warnings
warnings.filterwarnings("ignore")

from matplotlib import pyplot as plt, rcParams as rc


%matplotlib inline
rc["figure.figsize"] = 10,6

import datetime
import datetime as dt

from sklearn.model_selection  import StratifiedKFold
from sklearn.decomposition import PCA

from iteration_utilities import duplicates, unique_everseen

import sys
from itertools import groupby
from operator import itemgetter
from timeit import timeit


## RandomOverSampler to handle imbalanced data
from imblearn.over_sampling import RandomOverSampler # over sampling method 2

Using TensorFlow backend.


In [59]:

#Algorithm
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.naive_bayes import MultinomialNB

from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

#preprocessing
from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import StandardScaler, normalize



from sklearn.model_selection import train_test_split

import lightgbm as lgb


#Metrics
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, recall_score, precision_score
from sklearn.metrics import confusion_matrix, classification_report, f1_score, precision_recall_fscore_support

from sklearn.pipeline import Pipeline

# GridSearchCV to find optimal min_samples_leaf
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

In [3]:
df = pd.read_csv("train_Wc8LBpr.csv")

df_test = pd.read_csv("test_VsU9xXK.csv")
submission = pd.read_csv("sample_submission_NoPBkjr.csv")

df10 = df.copy()
df.shape, df_test.shape, submission.shape

((131662, 14), (87395, 13), (87395, 2))

In [4]:
def get_percentage_miss_value(dataset):
    higher_miss_value_column = []
    miss_threshold_value = 50
    
    for i in dataset.columns:
        if dataset[i].isna().sum() > 1: 
            perectange_val = (dataset[i].isna().sum() / len(dataset)) * 100
            print("Column-> " , i, ", total no of missing value : ",dataset[i].isna().sum() , " & :         ", round(perectange_val,2) ," %")
                
            if(perectange_val > miss_threshold_value):
                higher_miss_value_column.append(i)
            
    print("\n\n\n")
    
    if higher_miss_value_column:
        print("Higher Missing values in Columns for Delete : ", higher_miss_value_column)
    else:
        print("There are no Higher Column Missing values in Dataset")

In [25]:
get_percentage_miss_value(df)

Column->  Type_of_Cab , total no of missing value :  20210  & :          15.35  %
Column->  Customer_Since_Months , total no of missing value :  5920  & :          4.5  %
Column->  Life_Style_Index , total no of missing value :  20193  & :          15.34  %
Column->  Confidence_Life_Style_Index , total no of missing value :  20193  & :          15.34  %
Column->  Var1 , total no of missing value :  71030  & :          53.95  %




Higher Missing values in Columns for Delete :  ['Var1']


In [6]:
# get_percentage_miss_value(df_test)

In [7]:
# df.info()

In [8]:
def check_cloumn_details_type_numberical(dataset):
    for i in dataset.columns:
        if (dataset[i].dtype == "int"):
            print("Columns name :  ",i  )
            
            print(dict(Counter(dataset[i])))
            print("*"*100)
            print("\n")
            
            
def check_cloumn_details_type_float(dataset):
    for i in dataset.columns:
        if (dataset[i].dtype == "float"):
            print("Columns name :  ",i  )
            
            print(dict(Counter(dataset[i])))
            print("*"*100)
            print("\n")
            

In [9]:
# check_cloumn_details_type_numberical(df)

In [10]:
# check_cloumn_details_type_float(df)

In [13]:
def visualize_histogram(dataset):
    # plot histogram
    plt.figure(figsize=(25, 9))  # figure size in ratio 16:9
    features = dataset.columns  # list of columns name
    for i, j in enumerate(features):
        plt.subplot(3, 3, i + 1)  # create subplot for histogram
        plt.title("Histogram of {}".format(j), fontsize=15)  # title of histogram

        bins = len(dataset[j].unique())  # bins for histogram
        plt.hist(dataset[j], bins=bins, rwidth=0.8, edgecolor="y", linewidth=2, )  # plot histogram

    plt.subplots_adjust(hspace=0.5)  # space between horixontal axes (subplots)

In [14]:
# visualize_histogram(df)

In [15]:
# corr_2_more_visualize(df)

In [16]:


# visualize_numberical_values(df)

In [17]:
#Label encoding
def convert_to_numerical_label_encoding(dataset):
    enc = LabelEncoder()
    for i in dataset.columns:
        if(dataset[i].dtype == "object"):
            dataset[i] = enc.fit_transform(dataset[i])
            
    return dataset

In [18]:
class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0] if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)
    


In [26]:
def majority_imbalanced_dataset(dataset, target_col):
    feature_columns = dataset.columns.tolist()
    feature_columns = [c for c in feature_columns if c not in [target_col]]

    X2_new = dataset[feature_columns]
    Y2_new = dataset[target_col]

    os =  RandomOverSampler(random_state=35)
    X_feature_variables , y_output = os.fit_sample(X2_new, Y2_new)
    
    X_feature_variables[target_col] = y_output
    
    X_feature_variables = X_feature_variables.sample(frac = 1).reset_index(drop = True)
    
    return X_feature_variables

In [20]:
def standardscaler_preprocessing(dataset_train, dataset_test, num_col):
    scaler = StandardScaler()
   
    dataset_train[num_col] = scaler.fit_transform(dataset_train[num_col])

    dataset_test[num_col] = scaler.transform(dataset_test[num_col])
    
    return dataset_train, dataset_test

In [27]:
df.columns

Index(['Trip_ID', 'Trip_Distance', 'Type_of_Cab', 'Customer_Since_Months',
       'Life_Style_Index', 'Confidence_Life_Style_Index', 'Destination_Type',
       'Customer_Rating', 'Cancellation_Last_1Month', 'Var1', 'Var2', 'Var3',
       'Gender', 'Surge_Pricing_Type'],
      dtype='object')

In [28]:
Counter(df.Surge_Pricing_Type)

Counter({2: 56728, 3: 47720, 1: 27214})

# <font color ='green' > EDA

In [29]:
# drop_col = ["Trip_ID" , "Var1", "Var2", "Var3"]

drop_col = ["Trip_ID", "Var1"  ]

In [30]:
df1 = df.drop(columns = drop_col)
df.shape , df1.shape

((131662, 14), (131662, 12))

In [31]:
df1_test = df_test.drop(columns = drop_col)
df_test.shape , df1_test.shape

((87395, 13), (87395, 11))

In [33]:
# df1["Type_of_Cab"] = df1["Type_of_Cab"].fillna("Unkown")
# df1["Confidence_Life_Style_Index"] = df1["Confidence_Life_Style_Index"].fillna("Unkown")

In [34]:
df2 = DataFrameImputer().fit_transform(df1.copy())


df2_test = DataFrameImputer().fit_transform(df1_test.copy())

In [35]:
df2.isna().sum().sum(), df2_test.isna().sum().sum()

(0, 0)

In [36]:
df3 = convert_to_numerical_label_encoding(df2.copy())
df2.shape, df3.shape

((131662, 12), (131662, 12))

In [37]:
df3_test = convert_to_numerical_label_encoding(df2_test.copy())
df2_test.shape, df3_test.shape

((87395, 11), (87395, 11))

In [38]:
Counter(df3.Surge_Pricing_Type)

Counter({2: 56728, 3: 47720, 1: 27214})

In [39]:

# df3 = majority_imbalanced_dataset(df3.copy(), "Surge_Pricing_Type")
# Counter(df4.Surge_Pricing_Type)

In [40]:
df3.shape

(131662, 12)

In [41]:
# scaling_col = ["Trip_Distance" , "Customer_Since_Months" , "Life_Style_Index", "Customer_Rating", "Var2", "Var3"]

scaling_col = ["Trip_Distance" , "Customer_Since_Months" , "Life_Style_Index", "Customer_Rating"]



# df4, df4_test = standardscaler_preprocessing(df3.copy(), df3_test.copy(), scaling_col)

In [42]:
df3.head()

,Trip_Distance,Type_of_Cab,Customer_Since_Months,Life_Style_Index,Confidence_Life_Style_Index,Destination_Type,Customer_Rating,Cancellation_Last_1Month,Var2,Var3,Gender,Surge_Pricing_Type
0,6.77,1,1.0,2.427690,0,0,3.90500,0,46,60,0,2
1,29.47,1,10.0,2.782450,1,0,3.45000,0,56,78,1,2
2,41.58,1,10.0,2.802064,1,4,3.50125,2,56,77,1,2
3,61.56,2,10.0,2.802064,1,0,3.45375,0,52,74,1,3
4,54.95,2,10.0,3.034530,1,0,3.40250,4,49,102,1,2


In [43]:
df3.describe()

,Trip_Distance,Type_of_Cab,Customer_Since_Months,Life_Style_Index,Confidence_Life_Style_Index,Destination_Type,Customer_Rating,Cancellation_Last_1Month,Var2,Var3,Gender,Surge_Pricing_Type
count,131662.000000,131662.000000,131662.000000,131662.000000,131662.000000,131662.000000,131662.000000,131662.000000,131662.000000,131662.000000,131662.000000,131662.000000
mean,44.200909,1.603340,6.016661,2.802064,1.006228,1.000144,2.849458,0.782838,51.202800,75.099019,0.713190,2.155747
std,25.522882,1.177128,3.544410,0.207760,0.734909,1.906857,0.980675,1.037559,4.986142,11.578278,0.452274,0.738164
min,0.310000,0.000000,0.000000,1.596380,0.000000,0.000000,0.001250,0.000000,40.000000,52.000000,0.000000,1.000000
25%,24.580000,1.000000,3.000000,2.687952,0.000000,0.000000,2.152500,0.000000,48.000000,67.000000,0.000000,2.000000
50%,38.200000,1.000000,6.016661,2.802064,1.000000,0.000000,2.895000,0.000000,50.000000,74.000000,1.000000,2.000000
75%,60.730000,2.000000,10.000000,2.912815,2.000000,1.000000,3.582500,1.000000,54.000000,82.000000,1.000000,3.000000
max,109.230000,4.000000,10.000000,4.875110,2.000000,13.000000,5.000000,8.000000,124.000000,206.000000,1.000000,3.000000


In [44]:
X = df3.iloc[:,:-1]

Y = df3.iloc[:,-1]

In [ ]:
model_xgb = XGBClassifier(max_depth=5, objective='multi:softmax', num_classes=3)
model_xgb.model_xgb(X, Y)
y_pred_final = model_xgb.predict(df3_test)


# lgbclf= LGBMClassifier(gamma= 0.5, learning_rate= 0.05,max_depth= 3, n_estimators= 100)
model_lgb= LGBMClassifier(n_estimators= 200, objective='multi:softmax', num_classes=3)
model_lgb.fit(X,Y)
y_pred_final = model_lgb.predict(df3_test)



model_catg= CatBoostClassifier(depth= 3,l2_leaf_reg= 1e-25,learning_rate= 0.07,loss_function= 'MultiClass',n_estimators= 200)
model_catg.fit(X,Y)
y_pred_final = model_catg.predict(df3_test)


In [56]:
submission_1 = submission.copy()

print(df_test.shape , submission.shape , len(y_pred_final))

submission_1["Surge_Pricing_Type"] = y_pred_final

print(submission_1["Surge_Pricing_Type"].values)


submission_1.to_csv('XGBoost_final_20.csv', index=False)

df_submission_1 = pd.read_csv('XGBoost_final_17.csv')

print(df_submission_1.shape)

df_submission_1.head(3)

(87395, 13) (87395, 2) 87395
[1 2 2 ... 1 2 2]
(87395, 2)


,Trip_ID,Surge_Pricing_Type
0,T0005689459,1
1,T0005689462,2
2,T0005689463,2


In [61]:
Counter(df.Surge_Pricing_Type)

Counter({2: 56728, 3: 47720, 1: 27214})

In [62]:
df3[0:87395].shape  , df3[87395 : ].shape

((87395, 12), (44267, 12))

In [64]:
X_feature_variables1 , y_output1 = majority_imbalanced_dataset(df3, "Surge_Pricing_Type")

In [65]:
df10 = X_feature_variables1.copy()

In [66]:
df10["Surge_Pricing_Type"] = y_output1

In [67]:
df10.head()

,Trip_Distance,Type_of_Cab,Customer_Since_Months,Life_Style_Index,Confidence_Life_Style_Index,Destination_Type,Customer_Rating,Cancellation_Last_1Month,Var2,Var3,Gender,Surge_Pricing_Type
0,6.77,1,1.0,2.427690,0,0,3.90500,0,46,60,0,2
1,29.47,1,10.0,2.782450,1,0,3.45000,0,56,78,1,2
2,41.58,1,10.0,2.802064,1,4,3.50125,2,56,77,1,2
3,61.56,2,10.0,2.802064,1,0,3.45375,0,52,74,1,3
4,54.95,2,10.0,3.034530,1,0,3.40250,4,49,102,1,2


In [ ]:
def get_accuracy(y_train_val , y_pred_val , dataset_type = "Default"):
    
    print(" Dataset type is : ", dataset_type)
    
    print("\n Accuracy Score     : ",round(accuracy_score(y_train_val, y_pred_val), 4) * 100)
    
#     print("\n precision_accuracy : ",round(precision_score(y_train_val, y_pred_val), 4) * 100)
 
#     print("\n recall_accuracy    : ",round(recall_score(y_train_val, y_pred_val), 4) * 100)
    
#     print("\n roc_auc_accuracy   : ",round(roc_auc_score(y_train_val, y_pred_val), 4) * 100)
    
#     print("\n f1_score_accuracy  : ",round(f1_score(y_train_val, y_pred_val), 4) * 100)
    
#     print("\n explained_variance  : ",round(explained_variance_score(y_train_val, y_pred_val), 4) * 100)
    
#     tn, fp, fn, tp = confusion_matrix(y_train_val, y_pred_val).ravel()
    
#     print("\n Confusion Matrix TN : ", tn, " FP : ", fp, " FN : ", fn, " TP : ", tp)
    print(confusion_matrix(y_train_val, y_pred_val))

In [68]:
df10.shape , df3.shape

((170184, 12), (131662, 12))

In [69]:
new_test = df10[0:87395]

new_train = df10[87395 : ] 

In [70]:
x1_train = new_train.iloc[:, :-1]
y1_train = new_train.iloc[:, -1]

In [71]:
x1_test = new_test.iloc[:, :-1]
y1_test = new_test.iloc[:, -1]

In [77]:
model_final = XGBClassifier(max_depth=5, objective='multi:softmax', num_classes=3)

model_final.fit(x1_train, y1_train)

In [78]:
y_pred_final_new_test = model_final.predict(x1_test)
get_accuracy(y1_test , y_pred_final_new_test ,  "Test")

 Dataset type is :  Test

 Accuracy Score     :  58.26
[[14871  1772  1419]
 [14396 14692  8625]
 [ 4763  5500 21357]]


In [79]:
y_pred_final_new_train = model_final.predict(x1_train)
get_accuracy(y1_train , y_pred_final_new_train ,  "Train")

 Dataset type is :  Train

 Accuracy Score     :  69.41000000000001
[[32209  3577  2880]
 [ 7132  7910  3973]
 [ 3632  4130 17346]]


In [158]:
submission_new["Surge_Pricing_Type"] = y_pred_final_new_test


submission_new.to_csv('submission_new_imbalance.csv', index=False)
